In [4]:
import torch
import torchvision
from torchvision import transforms, models
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import cv2, os, json
import time, random
import matplotlib.pyplot as plt

#### 1. Creating the clips

* Firstly, the timestamps are manually marked and kept in a text-file, with a corresponding url of the yt video. The video is then downloaded locally.
* The downloaded video is loaded, then a clip of 3 seconds, 2s before the timestamp and 1s after, is created and stored in `data/`

In [ ]:
from pytubefix import YouTube
from pytubefix.cli import on_progress


# download the yt videos

video1_url = 'www.youtube.com/watch?v=P22HqM9w500'
video2_url = "https://www.youtube.com/watch?v=GsLhzD72yYA"
video3_url = 'https://www.youtube.com/watch?v=k4uLbGkm6Ls'
video4_url = "https://www.youtube.com/watch?v=xjy-7ZCohG8"

yt = YouTube(video4_url, on_progress_callback=on_progress).streams[0]


print(yt)
yt.download('video4.mp4')

In [ ]:
# load the timestamps and convert them from MM:SS formats to only seconds
with open("video4_kill_timestamps.txt") as f:
    lines = f.readlines()

lines = [t[:-1] for t in lines]
lines = [t.split(':') for t in lines]
lines = [int(t[0])*60+int(t[1]) for t in lines]

lines

In [ ]:
import moviepy as mp

#load the downloaded video
video = mp.VideoFileClip('video4.mp4')

#clip out the t-2 to t+1 section for each timestamp t, and save it
for t in lines:
    t = int(t)
    subclip = video[t-2:t+1]
    subclip.write_videofile(f"data/video4_{t-2}_{t+1}.mp4", codec='libx264')

#### function for loading the clips as frames

In [5]:
def load_video_as_frames(path:str):

    video = cv2.VideoCapture(path)
    success, img = video.read()

    
    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    frames = torch.zeros((frame_count, *img.shape))

    i= 0
    while success:

        frames[i] = torch.Tensor(img)
        i+=1
        success, img = video.read()

    
    frames = torch.permute(frames, (0, 3, 1, 2))
    
    return frames

    # torch.Tensor(frames[0])
    # print(frames.size())

### 2. The architechture(or pipeline, or whatver)

* Firstly, each clip is loaded, then it is batched and a feature representation is obtained for each individual frame. A pretrained model, (such as a resnet) is used for this, while optionally unfreezing the last layer 
* The features are then passed sequentially into an LSTM one after another, a single vector is finally recieved, encapsulating the sequential form of the data.
* now it is passed into a (Variational) autoencoder, and trained. 


In [13]:
from torchvision.models import resnet50, ResNet50_Weights

class myResnet(nn.Module):

    def __init__(self):
        super().__init__()

        self.model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
        self.transforms = ResNet50_Weights.IMAGENET1K_V2.transforms()

        self.model.fc = nn.Identity()

        for p in self.model.parameters():
            p.requires_grad_(False)
        
    def preprocess(self, X):
        return self.transforms(X)
    
    def forward(self, X):
        return self.model(X)
    
class myLSTM(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers):
        super().__init__()

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        # self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, X):

        out, (hn, cn) = self.lstm(X)

        #out.shape = (batch_size, seq_length, hidden_size)
        
        #take the last element from the sequence
        if out.ndim == 3: #batched
            out = out[:, -1, :] 
        elif out.ndim == 2: #unbatched
            out = out[-1, :]


        return out
    
class AE(nn.Module):

    def __init__(self, input_size, latent_size, hidden_dims:list=[256, 128]):
        
        super().__init__()
        layers = []

        hidden_dims.append(latent_size)
        hidden_dims.insert(0, input_size)

        for i in range(len(hidden_dims)-1):
            layers.append(nn.Linear(hidden_dims[i], hidden_dims[i+1]))
            layers.append(nn.ReLU())
        
        self.encoder = nn.Sequential(*layers)

        layers = []
        hidden_dims.reverse()
        for i in range(len(hidden_dims)-1):
            layers.append(nn.Linear(hidden_dims[i], hidden_dims[i+1]))
            layers.append(nn.ReLU())
        
        self.decoder = nn.Sequential(*layers)

    def forward(self, X):
        out = self.encoder(X)
        out = self.decoder(out)
        return out
    
    def get_loss(self, input, output):
        return F.mse_loss(input, output)

class VAE(nn.Module):

    def __init__(self, input_dim, latent_dim, hidden_dims=[256, 128]):
        super().__init__()

        layers = []

        k=input_dim

        for dim in hidden_dims:
            layers.append(nn.Linear(k, dim))
            layers.append(nn.LeakyReLU())

            k = dim

        self.encoder = nn.Sequential(*layers)

        self.fc_mu = nn.Linear(k, latent_dim)
        self.fc_log_var = nn.Linear(k, latent_dim)

        layers = []
        hidden_dims.reverse()
        k = latent_dim

        for dim in hidden_dims:
            layers.append(nn.Linear(k, dim))
            layers.append(nn.LeakyReLU())
            k = dim
        
        layers.append(nn.Linear(k, input_dim))

        self.decoder = nn.Sequential(*layers)
    
    def forward(self, X):

        out = self.encoder(X)

        #get mean and log(var)
        mean = self.fc_mu(out)
        log_var = self.fc_log_var(out)

        #reparametrization
        epsilon = torch.randn_like(mean)
        z = mean + epsilon*log_var

        #decode
        X_recon = self.decoder(z)

        return X_recon, mean, log_var
    
    def get_loss(self, X, target):

        X_recon, mean, log_var = target

        normal_loss = F.mse_loss(X, X_recon, reduce='sum') / X.shape[0]

        kld_loss = -0.5 * torch.mean(1+log_var-mean**2-log_var.exp()) / X.shape[0]

        return kld_loss + normal_loss


class myGRU(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers):
        super().__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)

    def forward(self, X):

        out, h_n = self.gru(X)

        #take last element from the sequence. 
        #out.shape = (batch_size, seq_len, hidden_size)

        if out.ndim == 3: #batched
            out = out[:, -1, :] 
        elif out.ndim == 2: #unbatched
            out = out[-1, :]

        return out

        

class MyModel(nn.Module):

    def __init__(self):
        super().__init__()

        ## model for getting feature encodings
        self.cnn_model = myResnet()
        
        ## LSTM (can also use attention based here if needed)
        

        self.lstm = myGRU(2048, 1024, 1)
        
        ## VAE

        self.ae = VAE(1024, 128, [256, 128])


    def forward(self, X):
        
        # assumes that X is an input of shape (seq_length, channels, height, width)

        encodings = self.cnn_model(X)
        #encoding has shape (seq_length, 2048), note that this is unbatched

        lstm_encoding = self.lstm(encodings)
        #this output has shape (1024, )

        out = self.ae(lstm_encoding)
     
        return lstm_encoding, out #both are necessary for loss
    

    def preprocess(self, X):
        return self.cnn_model.preprocess(X)
    
    def get_loss(self, lstm_encoding, output):

        return self.ae.get_loss(lstm_encoding, output)



        





In [14]:
sum([p.numel() for p in MyModel().parameters() if p.requires_grad])

10084352

In [16]:
with open("train_test_clips.json") as f:
    train_test_clips = json.load(f)

train_clips = train_test_clips['train']
test_clips = train_test_clips['test']



In [15]:
model = MyModel()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)
optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)


### Training the Model

In [ ]:
random.seed(0)

iteration_losses = []
epoch_losses = []

for epoch in range(7):
    
    total_loss = 0.
    start = time.time()

    
    random.shuffle(train_clips)

    iteration = 0
    for clip_path in train_clips:
        iteration += 1
        clip = load_video_as_frames(clip_path)

        # print(clip.shape)
        clip = model.preprocess(clip).to(device)

        ae_in, ae_out = model(clip)
        loss = model.get_loss(ae_in, ae_out)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss
        if (iteration+1)%10==0:
            print(f"loss for iteration{iteration+1} = {loss.item()}")
            iteration_losses.append(loss.item())


    epoch_losses.append(total_loss/len(train_clips))
    print(f"Epoch[{epoch}], avg_loss={total_loss/len(train_clips)}, time-taken={time.time()-start}")




In [7]:
torch.save(model.state_dict(), "mymodel_weights.pth")

In [ ]:
start = time.time()
# for clip_path in train_clips:
#         # print(clip_path)
#         clip = load_video_as_frames("./data/"+clip_path)

#         clip = model.preprocess(clip).to(device)

clip = load_video_as_frames("./data/video3_467_470.mp4").to(device)
clip = model.preprocess(clip).to(device)[0:]
print(clip.shape)
ae_in, ae_out = model(clip)
# print(ae_in.shape)
# print(ae_out.shape)


time.time()-start

torch.Size([90, 3, 224, 224])


8.781089544296265

In [ ]:
with torch.no_grad():

    losses = []

    for clip_path in test_clips:
        print(clip_path)
        clip = load_video_as_frames("./data/"+clip_path)
        clip = model.preprocess(clip).to(device)

        ae_in, ae_out = model(clip)
        loss = model.get_loss(ae_in, ae_out)

        losses.append(loss.item())

    print(losses)

    print(sum(losses)/len(losses))

In [ ]:
aimbot_clips = os.listdir("./aimbot_clips")

with torch.no_grad():

    losses = []

    for clip_path in aimbot_clips:
        print(clip_path, end=" ")
        clip = load_video_as_frames("./aimbot_clips/"+clip_path)
        clip = model.preprocess(clip).to(device)

        ae_in, ae_out = model(clip)
        loss = model.get_loss(ae_in, ae_out)

        print(loss.item())
        losses.append(loss.item())

    # print(losses)

    print(sum(losses)/len(losses))


In [ ]:
aimbot_clips = os.listdir("./aimbot_clips")

with torch.no_grad():

    losses = []

    for clip_path in aimbot_clips:
        print(clip_path, end=" ")
        clip = load_video_as_frames("./aimbot_clips/"+clip_path)
        clip = model.preprocess(clip).to(device)

        ae_in, ae_out = model(clip)
        loss = model.get_loss(ae_in, ae_out)

        print(loss.item())
        losses.append(loss.item())

    # print(losses)

    print(sum(losses)/len(losses))


In [ ]:
with torch.no_grad():

    losses=[]

    for t in lines:
        t = int(t)
        clip_path = f"data/video4_{t-2}_{t+1}.mp4"
        clip = load_video_as_frames(clip_path)
        clip = model.preprocess(clip).to(device)

        ae_in, ae_out = model(clip)
        loss = model.get_loss(ae_in, ae_out)

        print(clip_path, " ", loss.item())
        losses.append(loss.item())
    
    print(sum(losses)/len(losses))